In [1]:
import sys
sys.path.append('..')
from d2l import torch as d2l
import torch
import numpy as np

def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [2]:
X = torch.tensor([np.arange(9).reshape((3,3)), np.arange(1, 10).reshape((3,3))])
K = torch.tensor([np.arange(4).reshape((2,2)), np.arange(1, 5).reshape((2,2))])
X, K

(tensor([[[0, 1, 2],
          [3, 4, 5],
          [6, 7, 8]],
 
         [[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]], dtype=torch.int32),
 tensor([[[0, 1],
          [2, 3]],
 
         [[1, 2],
          [3, 4]]], dtype=torch.int32))

In [3]:
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [4]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [5]:
K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [6]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [7]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i), h*w)
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [8]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(d2l.reduce_sum(torch.abs(Y1- Y2))) < 1e-6, "not equal"